In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from ipywidgets import interact
import scipy.stats as stats
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
import matplotlib.pyplot as plt

import warnings # the warning spam is pointless and annoying
warnings.simplefilter(action="ignore", category=FutureWarning)

Populating the interactive namespace from numpy and matplotlib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_to_data="/content/drive/MyDrive/filtered_headlines.csv"
df=pd.read_csv(path_to_data, header=0 )
df

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn,Clean_Headline,Language,Processed_Headline,Topic 0,Topic 1,Topic 2,Topic 3
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1,Obama Lays Wreath at Arlington National Cemete...,en,obama lay wreath arlington nation cemeteri pre...,0.017137,0.016772,0.949220,0.016871
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1,"Tim Haywood, investment director business-unit...",en,tim haywood invest director busi unit head fix...,0.015689,0.724651,0.016840,0.242821
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1,"Nouriel Roubini, NYU professor and chairman at...",en,nouriel roubini nyu professor chairman roubini...,0.526804,0.436595,0.018314,0.018287
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1,Finland's economy expanded marginally in the t...,en,finland economi expand margin three month end ...,0.014203,0.957587,0.014114,0.014096
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1,Tourism and public spending continued to boost...,en,tourism public spend continu boost economi jan...,0.014868,0.955129,0.015017,0.014987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86280,61851.0,Stocks rise as investors key in on US economy ...,The June employment report is viewed as a cruc...,MarketWatch,economy,2016-07-07 15:31:05,0.104284,0.044943,-1,3,5,The June employment report is viewed as a cruc...,en,june employ report view crucial gaug health ec...,0.018213,0.945057,0.018308,0.018422
86281,61865.0,Russian PM proposes to use conservative and to...,"In addition, establish stimulating economic po...",TASS,economy,2016-07-07 15:31:10,0.072194,0.000000,-1,0,1,"In addition, establish stimulating economic po...",en,addit establish stimul econom polici budget co...,0.131143,0.820171,0.024560,0.024126
86282,104793.0,Palestinian Government Uses Foreign Aid To Pay...,The Palestinian government spends nearly $ mil...,Daily Caller,palestine,2016-07-07 15:38:26,0.291667,-0.139754,5,1,0,The Palestinian government spends nearly $ mil...,en,palestinian govern spend near million annual s...,0.518430,0.444394,0.019046,0.018131
86283,104794.0,Palestine Youth Orchestra prepares for first U...,Palestine Youth Orchestra prepares for first U...,Ahram Online,palestine,2016-07-07 15:59:22,0.121534,0.092313,0,0,0,Palestine Youth Orchestra prepares for first U...,en,palestin youth orchestra prepar first uk tour ...,0.125082,0.016564,0.398295,0.460058


In [4]:
#df = pd.read_csv("/Users/nuzhatnazmulnishi/Desktop/Winter 2024/MATH 509/Project/20240306/filtered_headlines.csv", header=0)

In [5]:
df = df[["IDLink", "Topic", "Facebook", "GooglePlus", "LinkedIn", "Topic 0", "Topic 1", "Topic 2", "Topic 3"]]
# df = df.loc[(df["Facebook"] != -1) & (df["GooglePlus"] != -1) & (df["LinkedIn"] != -1)]
#df = df.loc[(df["Facebook"] > 1) & (df["GooglePlus"] > 1) & (df["LinkedIn"] > 1)]
df = df.loc[(df["Facebook"] > 0) & (df["GooglePlus"] > 0) & (df["LinkedIn"] > 0)]

df["Facebook"] = stats.boxcox(df["Facebook"])[0]
df["GooglePlus"] = stats.boxcox(df["GooglePlus"])[0]
df["LinkedIn"] = stats.boxcox(df["LinkedIn"])[0]

obama = tf.cast(df["Topic 0"], tf.float32)
econ = tf.cast(df["Topic 1"], tf.float32)
pal = tf.cast(df["Topic 2"], tf.float32)
ms = tf.cast(df["Topic 3"], tf.float32)

fb = tf.cast(df["Facebook"], tf.float32)
gp = tf.cast(df["GooglePlus"], tf.float32)
li = tf.cast(df["LinkedIn"], tf.float32)

In [6]:
df

,IDLink,Topic,Facebook,GooglePlus,LinkedIn,Topic 0,Topic 1,Topic 2,Topic 3
690,292.0,economy,3.028515,0.631480,2.593960,0.007695,0.873518,0.007711,0.111076
692,328.0,microsoft,2.254225,0.000000,0.000000,0.018235,0.512172,0.017372,0.452221
696,476.0,microsoft,1.337438,0.631480,1.279743,0.044200,0.043614,0.044327,0.867860
700,15.0,economy,4.281484,0.631480,3.199247,0.018618,0.942860,0.019751,0.018770
701,16.0,economy,1.970810,0.949091,1.616472,0.017003,0.949058,0.017073,0.016867
...,...,...,...,...,...,...,...,...,...
86151,104737.0,obama,4.864450,0.949091,0.000000,0.197363,0.014886,0.772370,0.015380
86163,104789.0,obama,1.710709,0.631480,1.030959,0.078233,0.018432,0.884313,0.019022
86170,104744.0,obama,6.635342,2.504711,3.187751,0.193664,0.017395,0.772415,0.016526
86240,104777.0,obama,3.751837,0.000000,0.000000,0.017590,0.018037,0.946934,0.017440


In [7]:
print(df.shape)
mean_value = df.iloc[:, -4:].mean()
std_value = df.iloc[:, -4:].std()
print('Mean:',mean_value)
print('STD:', std_value)

mean_fb = df['Facebook'].mean()

print("Mean of the column FB:", mean_fb)

(22675, 9)
Mean: Topic 0    0.175138
Topic 1    0.251679
Topic 2    0.259473
Topic 3    0.313710
dtype: float64
STD: Topic 0    0.279692
Topic 1    0.326258
Topic 2    0.339799
Topic 3    0.376023
dtype: float64
Mean of the column FB: 3.543413483856451


## Generative Model
\begin{align*}
    Facebook_{i} &\sim \text{Normal}(\mu_{i}, \sigma) \\
    \mu_{i} &= \alpha + \beta_1* X_1 + \beta_2 * X_2 + \beta_3 * X_3 \\
    \alpha &\sim \chi^2 (1) \\
    \beta_1 & \sim \text{Normal}(0.170898, 0.276186) \\
    \beta_2 & \sim \text{Normal}(0.239065, 0.318835) \\
    \beta_3 & \sim \text{Normal}(0.263155, 0.341834) \\
    \sigma &\sim \text{Uniform}(0, 50)
\end{align*}


In [8]:
np.random.seed(8927)

Nsteps = 10000
N_burn_in = Nsteps
Nchains = 4

## FaceBook

In [9]:
#Obama = tf.math.log(tf.cast(df['Topic 0'],tf.float32))
#Econ = tf.math.log(tf.cast(df['Topic 1'],tf.float32))
#Palistine = tf.math.log(tf.cast(df['Topic 2'],tf.float32))

Obama = tf.cast(df['Topic 0'],tf.float32)
Econ = tf.cast(df['Topic 1'],tf.float32)
Palistine = tf.cast(df['Topic 2'],tf.float32)

FaceBook = tf.cast(df['Facebook'],tf.float32)

In [10]:
#def custom_log(df):
 #   facebook_plus_one = df['Facebook'] + 1
 #   log_value = tf.math.log(tf.cast(facebook_plus_one, tf.float32))
 #   return log_value


In [11]:
def joint_log_prob(alpha, beta1, beta2, beta3, sigma):
    # Define priors
 #   alpha = tf.cast(alpha, tf.float64)
  #  beta1 = tf.cast(beta1, tf.float64)
   ##beta3 = tf.cast(beta3, tf.float64)
    #sigma = tf.cast(sigma, tf.float64)


 #   alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
 #   beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.203525, tf.float64), scale=tf.cast(0.298182, tf.float64)).log_prob(beta1))
 #   beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.290288, tf.float64), scale=tf.cast(0.344120, tf.float64)).log_prob(beta2))
 #   beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.258918, tf.float64), scale=tf.cast(0.338090, tf.float64)).log_prob(beta3))
 #   sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float64), high=tf.cast(50.0, tf.float64)).log_prob(sigma))

    alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
    beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.170898, tf.float32), scale=tf.cast(0.276186, tf.float32)).log_prob(beta1))
    beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.239065, tf.float32), scale=tf.cast(0.318835, tf.float32)).log_prob(beta2))
    beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.263155, tf.float32), scale=tf.cast(0.341834, tf.float32)).log_prob(beta3))
    sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float32), high=tf.cast(50.0, tf.float32)).log_prob(sigma))

    mu = alpha + beta1_prior * Obama + beta2_prior  * Econ + beta3_prior  * Palistine

    _density_fb= tfd.Normal(loc=mu, scale=sigma).log_prob(FaceBook )
    #_density_fb= tfd.Exponential(1/mu).log_prob(FaceBook)
    density_fb = tf.reduce_sum(_density_fb, axis=0)

    # Sum of log probabilities
    return density_fb + alpha_prior + beta1_prior + beta2_prior + beta3_prior + sigma



In [12]:
# Define bijectors for transformations
bijectors = [
    tfp.bijectors.Square(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.SoftClip(low=0.0,high=50.0)
]

# Initial state for the sampler
initial_state = [
    tf.zeros([], dtype=tf.float32, name='init_alpha'),
    tf.zeros([], dtype=tf.float32, name='init_beta1'),
    tf.zeros([], dtype=tf.float32, name='init_beta2'),
    tf.zeros([], dtype=tf.float32, name='init_beta3'),
    tf.ones([], dtype=tf.float32, name='init_sigma')
]

In [13]:
@tf.function()
def sample_chain(initial_state):
    theta = tfp.mcmc.sample_chain(
        #num_results=Nsteps,
        num_results=2000,
        num_burnin_steps=N_burn_in,
        # num_burnin_steps=1000,
        current_state=initial_state,
        kernel=tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(

                # NUTS instead of HMC
                inner_kernel=tfp.mcmc.NoUTurnSampler(
                    target_log_prob_fn=joint_log_prob,
                    max_tree_depth=8,
                    max_energy_diff=1000,

                    step_size=tf.cast(0.01, tf.float32)),    # STEPSIZE TUNE

                bijector= bijectors ),

            num_adaptation_steps=600),
        trace_fn=None)
    return theta


In [14]:
alpha = chisquare(1)
beta1 = normal(0.170898, 0.276186)
beta2 = normal(0.2390656, 0.318835)
beta3 = normal(0.263155,0.341834)
sigma = uniform(0.0, 50.0)

theta = sample_chain([alpha, beta1, beta2, beta3, sigma])
print(theta)

[<tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([3.6141715, 3.6282747, 3.6168447, ..., 3.6247795, 3.6236234,
       3.6087031], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.1771607 , 0.17836119, 0.18381275, ..., 0.17564487, 0.17546244,
       0.19326258], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([-0.19558778, -0.20558341, -0.19505319, ..., -0.19553122,
       -0.19345015, -0.19287294], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.25689134, 0.26996794, 0.24313939, ..., 0.2569624 , 0.24488044,
       0.24312702], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([1.4651299, 1.4531326, 1.4571495, ..., 1.4626923, 1.4568253,
       1.4561462], dtype=float32)>]


In [15]:
# Extract samples
posterior = [theta[0].numpy(), theta[1].numpy(), theta[2].numpy(), theta[3].numpy(), theta[4].numpy()]

# Create a DataFrame
posterior_df = pd.DataFrame(posterior).T  # Transpose the DataFrame

# Set column names
posterior_df.columns = ['alpha', 'beta1', 'beta2', 'beta3', 'sigma']
print(posterior_df)

         alpha     beta1     beta2     beta3     sigma
0     3.614172  0.177161 -0.195588  0.256891  1.465130
1     3.628275  0.178361 -0.205583  0.269968  1.453133
2     3.616845  0.183813 -0.195053  0.243139  1.457150
3     3.627980  0.174668 -0.201638  0.271537  1.448883
4     3.620311  0.182736 -0.193245  0.254309  1.471901
...        ...       ...       ...       ...       ...
1995  3.623913  0.174691 -0.196456  0.248107  1.466915
1996  3.632840  0.156847 -0.202353  0.269358  1.460712
1997  3.624779  0.175645 -0.195531  0.256962  1.462692
1998  3.623623  0.175462 -0.193450  0.244880  1.456825
1999  3.608703  0.193263 -0.192873  0.243127  1.456146

[2000 rows x 5 columns]


## LinkedIn

In [16]:
Obama = tf.cast(df['Topic 0'],tf.float32)
Econ = tf.cast(df['Topic 1'],tf.float32)
Palistine = tf.cast(df['Topic 2'],tf.float32)

#LinkedIn = tf.math.log(tf.math.log(tf.cast(df['LinkedIn'],tf.float32)))
LinkedIn = tf.cast(df['LinkedIn'],tf.float32)

In [17]:
#def custom_log(df):
 #   li_plus_one = df['LinkedIn'] + 1
 #   log_value = tf.math.log(tf.cast(li_plus_one, tf.float32))
 #   return log_value

In [18]:
def joint_log_prob(alpha, beta1, beta2, beta3, sigma):
    # Define priors

    alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
    beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.170898, tf.float32), scale=tf.cast(0.276186, tf.float32)).log_prob(beta1))
    beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.239065, tf.float32), scale=tf.cast(0.318835, tf.float32)).log_prob(beta2))
    beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.263155, tf.float32), scale=tf.cast(0.341834, tf.float32)).log_prob(beta3))
    sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float32), high=tf.cast(50.0, tf.float32)).log_prob(sigma))


    mu = alpha + beta1_prior * Obama + beta2_prior  * Econ + beta3_prior  * Palistine

    _density_li = tfd.Normal(loc=mu, scale=sigma).log_prob(LinkedIn)
    density_li = tf.reduce_sum(_density_li, axis=0)

    # Sum of log probabilities
    return density_li + alpha_prior + beta1_prior + beta2_prior + beta3_prior + sigma_prior


In [19]:
# Define bijectors for transformations
bijectors = [
    tfp.bijectors.Square(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.SoftClip(low=0.0,high=50.0)
]

# Initial state for the sampler
initial_state = [
    tf.zeros([], dtype=tf.float32, name='init_alpha'),
    tf.zeros([], dtype=tf.float32, name='init_beta1'),
    tf.zeros([], dtype=tf.float32, name='init_beta2'),
    tf.zeros([], dtype=tf.float32, name='init_beta3'),
    tf.ones([], dtype=tf.float32, name='init_sigma')
]

In [20]:
@tf.function()
def sample_chain(initial_state):
    theta = tfp.mcmc.sample_chain(
        #num_results=Nsteps,
        num_results=2000,
        num_burnin_steps=N_burn_in,
        # num_burnin_steps=1000,
        current_state=initial_state,
        kernel=tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(

                # NUTS instead of HMC
                inner_kernel=tfp.mcmc.NoUTurnSampler(
                    target_log_prob_fn=joint_log_prob,
                    max_tree_depth=8,
                    max_energy_diff=1000,

                    step_size=tf.cast(0.01, tf.float32)),    # STEPSIZE TUNE

                bijector= bijectors ),

            num_adaptation_steps=600),
        trace_fn=None)
    return theta


In [21]:
alpha = chisquare(1)
beta1 = normal(0.170898, 0.276186)
beta2 = normal(0.2390656, 0.318835)
beta3 = normal(0.263155,0.341834)
sigma = uniform(0.0, 50.0)

theta = sample_chain([alpha, beta1, beta2, beta3, sigma])
print(theta)



[<tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([2.21841  , 2.21841  , 2.2259676, ..., 2.2047932, 2.2349248,
       2.1930978], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.61295116, 0.61295116, 0.62638354, ..., 0.6150799 , 0.6175606 ,
       0.61409515], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.07313991, 0.07313991, 0.07053521, ..., 0.0743181 , 0.05961652,
       0.0916542 ], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([-0.18757541, -0.18757541, -0.17720306, ..., -0.18432128,
       -0.17884472, -0.18275878], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([1.2220764, 1.2220764, 1.2284966, ..., 1.224205 , 1.2274475,
       1.2153435], dtype=float32)>]


In [22]:
# Extract samples
posterior = [theta[0].numpy(), theta[1].numpy(), theta[2].numpy(), theta[3].numpy(), theta[4].numpy()]

# Create a DataFrame
posterior_df = pd.DataFrame(posterior).T  # Transpose the DataFrame

# Set column names
posterior_df.columns = ['alpha', 'beta1', 'beta2', 'beta3', 'sigma']
print(posterior_df)

         alpha     beta1     beta2     beta3     sigma
0     2.218410  0.612951  0.073140 -0.187575  1.222076
1     2.218410  0.612951  0.073140 -0.187575  1.222076
2     2.225968  0.626384  0.070535 -0.177203  1.228497
3     2.225968  0.626384  0.070535 -0.177203  1.228497
4     2.199061  0.606130  0.065717 -0.174676  1.220726
...        ...       ...       ...       ...       ...
1995  2.200801  0.617756  0.080938 -0.171834  1.222988
1996  2.224523  0.614742  0.071559 -0.178650  1.212811
1997  2.204793  0.615080  0.074318 -0.184321  1.224205
1998  2.234925  0.617561  0.059617 -0.178845  1.227448
1999  2.193098  0.614095  0.091654 -0.182759  1.215343

[2000 rows x 5 columns]


## GooglePlus

In [23]:
#Obama = tf.math.log(tf.cast(df['Topic 0'],tf.float32))
#Econ = tf.math.log(tf.cast(df['Topic 1'],tf.float32))
#Palistine = tf.math.log(tf.cast(df['Topic 2'],tf.float32))

Obama = tf.cast(df['Topic 0'],tf.float32)
Econ = tf.cast(df['Topic 1'],tf.float32)
Palistine = tf.cast(df['Topic 2'],tf.float32)

#GooglePlus = tf.math.log(tf.math.log(tf.cast(df['GooglePlus'],tf.float32)))
GooglePlus = tf.cast(df['GooglePlus'],tf.float32)

In [24]:
#def custom_log(df):
 #   gplus_plus_one = df['GooglePlus'] + 1
 #   log_value = tf.math.log(tf.cast(gplus_plus_one, tf.float32))
 #   return log_value

In [25]:
def joint_log_prob(alpha, beta1, beta2, beta3, sigma):
    # Define priors

    alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
    beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.170898, tf.float32), scale=tf.cast(0.276186, tf.float32)).log_prob(beta1))
    beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.239065, tf.float32), scale=tf.cast(0.318835, tf.float32)).log_prob(beta2))
    beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.263155, tf.float32), scale=tf.cast(0.341834, tf.float32)).log_prob(beta3))
    sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float32), high=tf.cast(500.0, tf.float32)).log_prob(sigma))

    mu = alpha + beta1_prior * Obama + beta2_prior  * Econ + beta3_prior  * Palistine

    _density_li = tfd.Normal(loc=mu, scale=sigma).log_prob(GooglePlus)
    density_li = tf.reduce_sum(_density_li, axis=0)

    # Sum of log probabilities
    return density_li + alpha_prior + beta1_prior + beta2_prior + beta3_prior + sigma_prior

In [26]:
# Define bijectors for transformations
bijectors = [
    tfp.bijectors.Square(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.SoftClip(low=0.0,high=500.0)
]

# Initial state for the sampler
initial_state = [
    tf.zeros([], dtype=tf.float32, name='init_alpha'),
    tf.zeros([], dtype=tf.float32, name='init_beta1'),
    tf.zeros([], dtype=tf.float32, name='init_beta2'),
    tf.zeros([], dtype=tf.float32, name='init_beta3'),
    tf.ones([], dtype=tf.float32, name='init_sigma')
]

In [27]:
@tf.function()
def sample_chain(initial_state):
    theta = tfp.mcmc.sample_chain(
        #num_results=Nsteps,
        num_results=2000,
        num_burnin_steps=N_burn_in,
        # num_burnin_steps=1000,
        current_state=initial_state,
        kernel=tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(

                # NUTS instead of HMC
                inner_kernel=tfp.mcmc.NoUTurnSampler(
                    target_log_prob_fn=joint_log_prob,
                    max_tree_depth=8,
                    max_energy_diff=1000,

                    step_size=tf.cast(0.01, tf.float32)),    # STEPSIZE TUNE

                bijector= bijectors ),

            num_adaptation_steps=600),
        trace_fn=None)
    return theta

In [28]:
alpha = chisquare(1)
beta1 = normal(0.170898, 0.276186)
beta2 = normal(0.2390656, 0.318835)
beta3 = normal(0.263155,0.341834)
sigma = uniform(0.0, 500.0)

theta = sample_chain([alpha, beta1, beta2, beta3, sigma])
print(theta)



[<tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([1.0697742, 1.0697742, 1.0697742, ..., 1.0697742, 1.0697742,
       1.0697742], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.34452203, 0.34452203, 0.34452203, ..., 0.34452203, 0.34452203,
       0.34452203], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([-0.03612836, -0.03612836, -0.03612836, ..., -0.03612836,
       -0.03612836, -0.03612836], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.22278158, 0.22278158, 0.22278158, ..., 0.22278158, 0.22278158,
       0.22278158], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.9651489, 0.9651489, 0.9651489, ..., 0.9651489, 0.9651489,
       0.9651489], dtype=float32)>]


In [29]:
# Extract samples
posterior = [theta[0].numpy(), theta[1].numpy(), theta[2].numpy(), theta[3].numpy(), theta[4].numpy()]

# Create a DataFrame
posterior_df = pd.DataFrame(posterior).T  # Transpose the DataFrame

# Set column names
posterior_df.columns = ['alpha', 'beta1', 'beta2', 'beta3', 'sigma']
print(posterior_df)

         alpha     beta1     beta2     beta3     sigma
0     1.069774  0.344522 -0.036128  0.222782  0.965149
1     1.069774  0.344522 -0.036128  0.222782  0.965149
2     1.069774  0.344522 -0.036128  0.222782  0.965149
3     1.069774  0.344522 -0.036128  0.222782  0.965149
4     1.069774  0.344522 -0.036128  0.222782  0.965149
...        ...       ...       ...       ...       ...
1995  1.069774  0.344522 -0.036128  0.222782  0.965149
1996  1.069774  0.344522 -0.036128  0.222782  0.965149
1997  1.069774  0.344522 -0.036128  0.222782  0.965149
1998  1.069774  0.344522 -0.036128  0.222782  0.965149
1999  1.069774  0.344522 -0.036128  0.222782  0.965149

[2000 rows x 5 columns]


In [30]:
sigma

441.3578826147642